In [21]:
import pandas as pd
import altair as alt

# Cargar archivo
df = pd.read_excel('base_pernoctaciones_tasa.xlsx')

import pandas as pd
import altair as alt

# Cargar archivo
df = pd.read_excel('base_pernoctaciones_tasa.xlsx')

# Filtrar fila "TOTAL CHILE"
df_region = df[df["Nombre región"].astype(str).str.strip().str.upper() == "VALPARAÍSO"]


# Definir años explícitos
años = [str(a) for a in range(2017, 2026)]

# Definir columnas exactas
cols_per = [f"Per {a}" for a in años]
cols_tasa = [f"Tasa {a}" for a in años]

# Extraer valores directamente (solo esos 9 años)
valores_per = df_region[cols_per].iloc[0].astype(str).str.replace(".", "", regex=False)
valores_tasa = df_region[cols_tasa].iloc[0].astype(str).str.replace(",", ".", regex=False)

# Convertir a numérico
pernoctaciones = pd.to_numeric(valores_per.values[:9], errors='coerce') / 10  # 👈 quitar un cero
tasa_ocupacion = pd.to_numeric(valores_tasa.values[:9], errors='coerce')

# Crear DataFrame final
data = pd.DataFrame({
    "Año": años,
    "Pernoctaciones": pernoctaciones,
    "Tasa_Ocupación": tasa_ocupacion
})
data["Pernoctaciones_millones"] = data["Pernoctaciones"] / 1_000_000
data = data[data["Año"] != "2025"]
# Crear el gráfico base
base = alt.Chart(data).encode(
    x=alt.X('Año:O', title='Año')
,   tooltip=[
        alt.Tooltip('Año:N'),
        alt.Tooltip('Pernoctaciones_millones:Q', title='Pernoctaciones (millones)', format='.1f'),
        alt.Tooltip('Tasa_Ocupación:Q', title='Tasa de ocupación (%)', format='.1f')
    ])


# Barras sin notación científica
barras = base.mark_bar(color='steelblue').encode(
    y=alt.Y('Pernoctaciones_millones:Q',
            title='Pernoctaciones (millones)',
            axis=alt.Axis(format='.1f', titleColor='steelblue'))  # <- sin notación científica
)
# Línea usando la tasa real
linea = base.mark_line(color='crimson', strokeWidth=5).encode(
    y=alt.Y('Tasa_Ocupación:Q',
            title='Tasa de ocupación (%)',
            axis=alt.Axis(format='.1f', titleColor='crimson'))
)

# Combinar ambos
chart = alt.layer(barras, linea).resolve_scale(
    y='independent'  # Permite que las barras y la línea tengan sus propias escalas
).properties(
    width=700,
    height=400,
    title='Pernoctaciones y Tasa de Ocupación Habitaciones Región de Valparaíso (2017–2024)'
)
chart
chart.save('grafico_pernoctaciones_valparaiso.html')
from google.colab import files
files.download('grafico_pernoctaciones_valparaiso.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>